# Figure DGVM carbon deficit maps

In [ ]:
# Libraries
import os
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
import geopandas as gpd
import cartopy.crs as ccrs

In [ ]:
# Directories
dir05x = '../paper_deficit/output/05_prep_other/fig_dgvm/'
dir06 = '../paper_deficit/output/06_eval/'
dir_nearth = '../data/naturalearth/'

---

In [ ]:
# Get coastline data
coastline110 = gpd.read_file(
    os.path.join(dir_nearth, 'ne_110m_coastline/ne_110m_coastline.shp'))

# Get data on primary land in 1700
ds_prim = xr.open_dataset(os.path.join(dir05x, 'luh2', 'ds_luh2_prim_1700.nc'),
                          chunks=dict(lat=5000, lon=5000))

In [ ]:
# Get and prepare dgvm data

def prep_dgvm(m):
    # Get dgvm data
    ds = xr.open_dataset(os.path.join(dir05x, m, f'ds_{m}.nc'),
                         chunks=dict(lat=5000, lon=5000))
    # Get primary land data at resolution of dgvm
    ds_prim_x = xr.open_dataset(os.path.join(dir05x, m, f'ds_{m}_prim.nc'),
                         chunks=dict(lat=5000, lon=5000))
    # Create dataset with deficit variables
    ds_out = xr.Dataset()
    ds_out['def_cveg'] = (ds[m + '_s2_cveg'] - ds[m + '_s3_cveg'])
    ds_out['def_csoil'] = (ds[m + '_s2_csoil'] - ds[m + '_s3_csoil'])
    # Multiply dgvm deficits with primary land in 1700 array
    ds_out = ds_out * ds_prim_x.prim_1700
    # Set crs
    ds_out.rio.write_crs("epsg:4326", inplace=True)
    # Add attributes
    ds_out.attrs['model'] = m.upper()
    # Adjust x axis if necessary
    if m in ['clm', 'isam', 'jules']:
        ds_out['lon'] = (ds_out['lon'] + 180) % 360 - 180
        ds_out = ds_out.sortby(ds_out.lon)
    
    return ds_out


ds_cablepop = prep_dgvm('cablepop').persist()
ds_classic = prep_dgvm('classic').persist()
ds_clm = prep_dgvm('clm').persist()
ds_dlem = prep_dgvm('dlem').persist()
ds_ibis = prep_dgvm('ibis').persist()
ds_isam = prep_dgvm('isam').persist()
ds_jsbach = prep_dgvm('jsbach').persist()
ds_jules = prep_dgvm('jules').persist()
ds_lpjguess = prep_dgvm('lpjguess').persist()
ds_lpjwsl = prep_dgvm('lpjwsl').persist()
ds_orchidee = prep_dgvm('orchidee').persist()

# Adjust attributes if ncessary
ds_cablepop.attrs['model'] = 'CABLE-POP'
ds_lpjguess.attrs['model'] = 'LPJ-GUESS'
ds_lpjwsl.attrs['model'] = 'LPJ-WSL'

In [ ]:
# Prepare data from random forest
def prep_pot():
    # Get predicted data
    ds = xr.open_zarr(os.path.join(dir05x, 'pot', 'ds_pot.zarr'),
                      chunks=dict(lat=5000, lon=5000))
    # Get primary land data
    ds_prim_x = xr.open_zarr(os.path.join(dir05x, 'pot', 'ds_pot_prim.zarr'),
                             chunks=dict(lat=5000, lon=5000))
    # Create dataset with deficit variables
    ds_out = xr.Dataset()
    ds_out['def_cveg'] = ds.pot_s2_cveg - ds.pot_s3_cveg
    ds_out['def_csoil'] = ds.pot_s2_csoil - ds.pot_s3_csoil

    # Multiply dgvm deficits with primary land in 1700 array
    ds_out = ds_out * ds_prim_x.prim_1700
    # Set crs
    ds_out.rio.write_crs("epsg:4326", inplace=True)
    # Add attributes
    ds_out.attrs['model'] = 'This study'
    
    return ds_out


ds_pot = prep_pot().persist()

In [ ]:
# Get dgvm data at luh2 resolution to calculate ensemble average
def get_dgvm_mean():
    # Create dataset with deficit variables
    ds2 = xr.Dataset()
    for m in ['cablepop', 'classic', 'clm', 'dlem', 'ibis', 'isam', 'jsbach', 
                 'jules', 'lpjguess', 'lpjwsl', 'orchidee']:
        ds = xr.open_dataset(os.path.join(dir05x, m, f'ds_{m}_luh2res.nc'))
        ds2[f'{m}_def_cveg'] = (ds[f'{m}_s2_cveg'] - ds[f'{m}_s3_cveg'])
        ds2[f'{m}_def_csoil'] = (ds[f'{m}_s2_csoil'] - ds[f'{m}_s3_csoil'])
    
    # Get luh2 primary land 1700 data
    ds_prim = xr.open_dataset(os.path.join(dir05x, 'luh2', f'ds_luh2_prim_1700.nc'))
    
    # Create dataset with mean deficit of dgvms and multiply with primary land in 1700 array
    ds_mean = xr.Dataset()
    for ctype in ['cveg', 'csoil']:
        ds_mean['def_' + ctype] = ds2[[i for i in ds2.data_vars if i.endswith(ctype)]] \
            .to_array().mean('variable') * ds_prim.prim_1700
    
    # Add attributes
    ds_mean.attrs['model'] = 'DGVM mean'

    return ds_mean
    

ds_mean = get_dgvm_mean().persist()

In [ ]:
# List of datasets of dgvms, ds_mean, and ds_pot
list_ds = [ds_mean,ds_cablepop, ds_classic, ds_clm, ds_dlem, ds_ibis, ds_isam, 
           ds_jsbach, ds_jules, ds_lpjguess, ds_lpjwsl, ds_orchidee, ds_pot]

In [ ]:
# Plot
def plot_dgvm(v, label, fstr_out, vmin, vmax):

    fig = plt.figure(figsize=(9, 9), dpi=600, constrained_layout=True)
    fig.set_facecolor('#ffffff')
    
    gs = fig.add_gridspec(5, 3,
                          hspace=0, wspace=0,
                          width_ratios=(.33, .33, .33), 
                          height_ratios=(.2, .2, .2, .2, .2))
    
    ax0 = fig.add_subplot(gs[0], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax1 = fig.add_subplot(gs[1], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax2 = fig.add_subplot(gs[2], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax3 = fig.add_subplot(gs[3], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax4 = fig.add_subplot(gs[4], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax5 = fig.add_subplot(gs[5], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax6 = fig.add_subplot(gs[6], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax7 = fig.add_subplot(gs[7], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax8 = fig.add_subplot(gs[8], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax9 = fig.add_subplot(gs[9], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax10 = fig.add_subplot(gs[10], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax11 = fig.add_subplot(gs[11], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax12 = fig.add_subplot(gs[12], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax13 = fig.add_subplot(gs[13], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    ax14 = fig.add_subplot(gs[14], projection=ccrs.Robinson(central_longitude=10), aspect='auto')
    
    axs = [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12, ax13, ax14]
    
    for i in range(0, 13):
        # Coastline data
        coastline110.plot(ax=axs[i], transform=ccrs.PlateCarree(), 
                          color='#000000', linewidth=0.5)
        # Dgvm data
        im0 = list_ds[i][v] \
            .plot.imshow(ax=axs[i], transform=ccrs.PlateCarree(),
                         vmin=vmin, vmax=vmax, cmap='coolwarm', add_colorbar=False)
        # Subplot settings
        axs[i].set_title(list_ds[i].attrs['model'])
        
        axs[i].set_extent((-180, 180, -60, 90), ccrs.PlateCarree())
        axs[i].axis('off')
    
    # Switch off axis of last two plots
    for i in [ax13, ax14]:
        i.axis('off')
    
    # Colorbar
    cbar_ax = fig.add_axes([0.4, 0.1, 0.55, 0.015])
    cbar0 = fig.colorbar(im0, cax=cbar_ax, 
                 orientation = 'horizontal', 
                 extend='both',
                 label = label + ': Deficit (tC ha$^{-1}$)', 
                 shrink = 0.6, aspect = 60, pad=0.1)
        
    cbar0.ax.tick_params(size=0)
    
    # Export
    plt.savefig(os.path.join(dir06, f'pdf/{fstr_out}.pdf'), dpi=600)
    plt.savefig(os.path.join(dir06, f'png/{fstr_out}.png'), dpi=600);

In [ ]:
plot_dgvm('def_cveg', 'Vegetation carbon', 'figs04_dgvm_carbon_deficit_maps_vegetation', -50, 50)

In [ ]:
plot_dgvm('def_csoil', 'Soil organic carbon', 'figs05_dgvm_carbon_deficit_maps_soil', -50, 50)